In [1]:
import pybullet as p
import pybullet_data
import numpy as np
import time

pybullet build time: Feb  4 2024 12:55:26


In [3]:
class TableTennisEnv:
    def __init__(self, show_gui = False):
        self.step_count = 0
        self.show_gui = show_gui
        self._plane = None
        self._surface_body_id = None
        self._table_body_id = None
        # self._table2_body_id = None
        self._robotic_arm = None
        self._ball = None
        self.state = self.init_state()

        # Throw the ball
        # self.throw_ball()
        # print(self.state)


    def init_state(self):
        if self.show_gui:
            p.connect(p.GUI)
        else:
            p.connect(p.DIRECT)

        p.resetSimulation()
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)

        # load the plane
        self._plane = p.loadURDF('plane.urdf')
        p.changeDynamics(self._plane, -1, lateralFriction = 0.5)

        #load the platform
        surface_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[5, 3, 0.1])
        surface_visual_id = -1  # No visual representation for now
        self._surface_body_id = p.createMultiBody(baseMass=300.0, baseCollisionShapeIndex=surface_id, baseVisualShapeIndex=surface_visual_id, basePosition=[0, 0, 0.1])
        p.changeDynamics(self._surface_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)


        # load the table for the tennis
        table_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[1.37, 0.75, 0.5])
        table_visual_id = -1  # No visual representation for now
        self._table_body_id = p.createMultiBody(baseMass=200.0, baseCollisionShapeIndex=table_id, baseVisualShapeIndex=table_visual_id, basePosition=[0, 0, 0.7])
        p.changeDynamics(self._table_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)


        # load the secondary table for the robot
        table2_id = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.5, 0.5, 0.5])
        table2_visual_id = -1  # No visual representation for now
        self._table2_body_id = p.createMultiBody(baseMass=50.0, baseCollisionShapeIndex=table2_id, baseVisualShapeIndex=table2_visual_id, basePosition=[2, 0, 0.7])
        p.changeDynamics(self._table2_body_id, -1, restitution=0.8, lateralFriction = 0.5)  # Adjust the restitution as needed (e.g., 0.8 for a good bounce)


        # load the robotic arm
        self._robotic_arm = p.loadURDF('TableTennisEnvironmentV0/combined_robot.urdf', basePosition=[1.7,0,1.25], baseOrientation=[0,0,0,1], globalScaling=1.0, useFixedBase=True)
        p.changeDynamics(self._robotic_arm, 9, restitution = 0.8)


        # set the arm position 
        for i in range(1000):
            joint_1 = i/500
            joint_2 = 3.0
            joint_3 = 3.0
            joint_4 = 3.0
            joint_5 = 3.0
            joint_6 = 1.5
            end_effector = 1 # useless
            finger1_joint1 = 1
            finger1_joint2 = 1
            finger2_joint1 = 1
            finger2_joint2 = 1
            finger3_joint1 = 1
            finger3_joint2 = 1
        
            joint_angles_vector = [0.0, 0.0, joint_1, joint_2, joint_3, joint_4, joint_5, joint_6,
                                  end_effector, finger1_joint1, finger1_joint2, 
                                  finger2_joint1, finger2_joint2,
                                  finger3_joint1, finger3_joint2,]
            
            p.setJointMotorControlArray(self._robotic_arm, range(15), 
                                        p.POSITION_CONTROL,
                                        targetPositions=joint_angles_vector)

            p.stepSimulation()

        # load the tennis ball
        self._ball = p.loadURDF('TableTennisEnvironmentV0/tennis_ball2/urdf/tennis_ball2.urdf', basePosition = [-1, 0, 2])
        p.changeDynamics(self._ball, -1, restitution = 0.8)
        # p.changeDynamics(self._ball, -1, mass = 0.02)
        # force_vector_ball = [3.05, 0.8, 2.4]
        # p.applyExternalForce(self._ball, -1, forceObj=force_vector_ball, posObj=[0, 0, 0], flags=p.LINK_FRAME)


        self.state = self.get_state()
        return self.state


    # get states at any time
    def get_state(self): 
        position_ball, orientation_ball = p.getBasePositionAndOrientation(self._ball)
        position_bat, orientation_bat, _, _, _, _ = p.getLinkState(self._robotic_arm, 9) ## it is the position of the end effector
        state = {
            'ball': {
                'position': position_ball,
                'orientation': orientation_ball,
            },
            'bat': {
                'position': position_bat,
                'orientation': orientation_bat,
            },
        }

        return state


    # reset the TableTennisEnvironmentV0 to initial config
    def reset(self):
        p.disconnect()
        self.state = self.init_state()
        self.step_count = 0


    def shutdown(self):
        p.disconnect()


    
    # calling this function will create a new ball 
    def get_new_ball(self, position = [-1, 0, 2]):
        # load the tennis ball
        self._ball = p.loadURDF('TableTennisEnvironmentV0/tennis_ball2/urdf/tennis_ball2.urdf', basePosition = position)
        p.changeDynamics(self._ball, -1, restitution = 0.8)
    
    # throw ball function --> velocity required : meters/sec
    def throw_ball(self, velocity_vector = [3.05, 0.8, 2.4]):  # set the force[force_x, force_y, force_z] # unit in Newton
        # p.applyExternalForce(self._ball, -1, forceObj=force_vector, posObj=[0, 0, 0], flags=p.LINK_FRAME)
        # p.stepSimulation()
        p.resetBaseVelocity(self._ball, velocity_vector)
        # self.state = self.get_state()


    # control joint velocity of the robot --> velocity is radians/sec
    def set_joint_velocity(self, target_velocities):
        for joint_index in range(2, 8):
            # Set the joint velocity for each joint
            p.setJointMotorControl2(bodyUniqueId=self._robotic_arm,
                                    jointIndex=joint_index,
                                    controlMode=p.VELOCITY_CONTROL,
                                    targetVelocity=target_velocities[joint_index-2],
                                    force=500)  # You can adjust the force parameter




    ## calculate the reward function
    def calc_reward(self):
        reward = -1
        return reward



    ## define the done condition
    def is_done(self):
        if self.step_count > 50000:
            return True
        else:
            return False



    ## step function
    def step(self, action): # action = [joint1, joint2, joint3, joint4, joint5, joint6] --> joint velocities
        self.step_count += 1
        joint_1 = action[0]
        joint_2 = action[1]
        joint_3 = action[2]
        joint_4 = action[3]
        joint_5 = action[4]
        joint_6 = action[5]
    
        joint_velocity_vector = [joint_1, joint_2, joint_3, joint_4, joint_5, joint_6]
        
        self.set_joint_velocity(joint_velocity_vector)


        # step
        p.stepSimulation()

        # update the state
        self.state = self.get_state()

        if self.is_done(): # setting done condition --> episode ends
            # set reward
            self.reset()
            reward = self.calc_reward()  # arbitrary reward currently, needs to be updated
            done = True
            return reward, done
        
        # if not done
        done = False
        reward = self.calc_reward() # arbitrary reward

        return reward, done

In [4]:
env = TableTennisEnv(show_gui = True)

2024-02-14 12:28:17.149 Python[65639:16896526] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


Version = 4.1 Metal - 88
Vendor = Apple
Renderer = Apple M2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [5]:
env.get_new_ball(position = [-2, 0, 2])

## Gather initial conditions
ball_initial_position = p.getBasePositionAndOrientation(env._ball)[0] # can be collected from optitrack
robot_initial_position = p.getLinkState(env._robotic_arm, 0)[0]      # can be collected from optitrack
distance_x, distance_y, distance_z = (robot_initial_position[0] - ball_initial_position[0], robot_initial_position[1] - ball_initial_position[1], robot_initial_position[2] - ball_initial_position[2])
print('distance_x: ', distance_x, 'm')


### define initial velocities [needs to be estimated by the optitrack]
u_x, u_y, u_z = 5.5, 0.8, -3     # in meters/sec
initial_velocity_ball = [u_x, u_y, u_z]

t_collision_estimated = distance_x / u_x   # estimated time for the ball to reach Y-Z plane
print('estimated_collision_time: ', t_collision_estimated, 'sec')


 
def ball_crossed_yz(ball_current_position, robot_current_position):#this function will detect if the ball reached Y-Z plane
    ball_current_position = p.getBasePositionAndOrientation(env._ball)[0] # can be collected from optitrack
    x_ball, y_ball, z_ball = ball_current_position
    x_robot, y_robot, z_robot = robot_current_position

    if x_ball > x_robot:  # this condition ensures the ball crossed Y-Z plane
        return True

    else:
        return False


def convert_step_to_time(step_number, frequency = 240):
    return step_number/frequency


    
env.throw_ball(initial_velocity_ball)

action = [0, 0, 0, 0, 0, 0] # keep the robot stopped, we don't need it.

for step in range(5000):
    env.step(action)
    ball_current_position = p.getBasePositionAndOrientation(env._ball)[0]
    if ball_crossed_yz(ball_current_position, robot_initial_position):
        print(convert_step_to_time(step, 240))
        break
    time.sleep(1/240) # because the simulation is at 240 Hz
    

# env.throw_ball([5.5, -0.8, -3])
# action = np.random.uniform([10, 0, 0, 0, 0, 0])
# print(action)
# reward, done = env.step(action)
# for step in range(5000):
#     bat_position = p.getLinkState(env._robotic_arm, 9)[0]
#     ball_position = p.getBasePositionAndOrientation(env._ball)[0]
#     distance = np.linalg.norm(np.array(bat_position) - np.array(ball_position))
#     print(distance, end = '\r')
#     # if step > 3500:
#     #     action = [0, 0, 0, 0, 0, 0]
#     reward, done = env.step(action)
#     time.sleep(1/240)
#     # print(env.state)

distance_x:  3.7005041600000004 m
estimated_collision_time:  0.6728189381818183 sec
1.225


In [6]:
p.getBaseVelocity(env._ball)

((1.7625584539477035, 0.05254170720793249, -1.8234148857517691),
 (-1.0209745201323293, 32.852923391662586, 1.5135537204160148))

In [32]:
bat_state = p.getLinkState(env._robotic_arm, 9)
bat_position = bat_state[0]
print(bat_position)

(1.3870412830461178, -0.1223690769560803, 1.5972621143671053)


In [33]:
env.get_new_ball(bat_position)

In [29]:
ball_state = p.getBasePositionAndOrientation(env._ball)
ball_position = ball_state[0]
print(ball_position)

(0.21409503739458796, -3.4836607673070836, 0.026382208037777473)
